<a href="https://colab.research.google.com/github/TanoOliva/ADA-Informes/blob/main/Informe_CPMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Descripcion del Problema

El Container Pre–Marshalling Problem **(CPMP)** es un problema de **optimización** que se sitúa en 
los terminal portuario, en el cual se almacenan una cierta cantidad de contenedores. El 
contexto de la ubicación de los contenedores, está dividido en bloques, y cada uno de estos 
consiste en varias bahías; y a su vez cada bahía posee un número determinado de pilas de 
contenedores. En la vida real, los días en los cuales los contenedores están en el puerto se 
tienen diversas restricciones, las cuales están en directa relación a la estabilidad de las pilas, el 
peso del contenedor así como el puerto al cual está destinado; lo anterior ha llevado a que se 
cree una numeración (prioridad) para grupos con el fin de referenciar a los contenedores que 
compartan los mismos atributos, con el fin de referirse de forma general a un grupo de 
contendores, para esta investigación la prioridad está dada únicamente por la urgencia que 
tendrá un contenedor de ser despachado. 


#2. Codigo

El siguiente codigo muestra la implementacion de CPMP


#### ESTRUCTURA DEL PROBLEMA class Layout:

In [157]:
#ESTRUCTURA DEL PROBLEMA
import random 

class Layout:
    def __init__(self, stacks, H):
        self.stacks = stacks
        self.sorted_elements = [] #for each stack

        self.steps = 0
        self.moves = []

        self.H = H
        self.G=max(set().union(*stacks))
        self.S=len(stacks)
        self.N = 0
        
        j=0
        
        for stack in stacks:
            self.N += len(stack)
            self.sorted_elements.append(compute_sorted_elements(stack))
            j += 1
    
    def __str__(self):
      _str = ""
      for k in range(len(self.stacks)):
        _str+= "stack "+ str(k)+": "+str(self.stacks[k])+"\n"
      return _str

    def is_valid_move(self, move):
      i = move[0]; j=move[1]
      if i==j: return False
      if len(self.stacks[i]) == 0: return False
      if len(self.stacks[j]) == self.H: return False

      return True

    def move(self,move):
        i = move[0]; j=move[1]
        
        if self.is_valid_move(move)==False: return None

               
        c = self.stacks[i][-1]

        if self.is_sorted(i): 
          self.sorted_elements[i] -= 1
            
        if self.is_sorted(j) and self.g(j) >= c:
            self.sorted_elements[j] += 1

            
        self.stacks[i].pop(-1)
        self.stacks[j].append(c)
        
        self.is_sorted(i)
        self.is_sorted(j)
        self.steps += 1
        self.moves.append((i,j))
        
        return c
                       
    def is_sorted(self, j):
        sorted = len(self.stacks[j]) == self.sorted_elements[j]
        return sorted

    def g(self, i):
        if len(self.stacks[i])==0: return self.G
        else: return self.stacks[i][-1]

    def bad_placed_elements(self):
      return self.N - sum(self.sorted_elements)
  
    def whereMax(self):
      max = 0
      for i in range(self.S):
        A = self.stacks[i]
        for j in range(len(A)):
          if A[j]> max:
            max = A[j]
            stackMax = i
            floorMax = j
      return stackMax,floorMax,max
    
    def isMaxTop(self):
      sMax,fMax,max = self.whereMax()
      if self.g(sMax) == max:
        return True
      else: return False
      
    def whereEmpty(self):
      a = []
      for i in range(self.S):
        A = self.stacks[i]
        if len(A) == 0:
          return i;
      return -1
    
    def emptyingStack(slef,notmove):
      i = random.randrange(0,lay.S)
      while i == notmove:#se busca un stack para vaciar aleatorio
        i = random.randrange(0,lay.S)
      while (len(lay.stacks[i]) != 0):#ciclo que se asegurará de que el stack quede vacío
        j = random.randrange(0,lay.S)
        while j == i or j == notmove: #se buscaran stacks aleatorios para depositar los contenedores del stack i
          j = random.randrange(0,lay.S)
        lay.move((i,j))
      return i
      

def compute_sorted_elements(stack):
    if len(stack)==0: return 0
    sorted_elements=1
    while(sorted_elements<len(stack) and stack[sorted_elements] <= stack[sorted_elements-1]):
        sorted_elements +=1
    
    return sorted_elements


#Genera unn layout aleatorio
def generate_random_layout(S,H,N):
    stacks = []
    for i in range(S):
        stacks.append([])
    for j in range(N):
        s=random.randint(0,S-1);
        while len(stacks[s])==H: s=s=random.randint(0,S-1);
        stacks[s].append(random.randint(1,N));
    return Layout(stacks,H)  


In [305]:
lay = generate_random_layout(5,5,7)
print(lay)
print(lay.whereMax())
greedy_random2(lay,10)


stack 0: []
stack 1: []
stack 2: [3, 6, 3, 7]
stack 3: [4]
stack 4: [3, 6]

(2, 3, 7)


(False, 2)

### Algoritmo para la resolucion del problema

In [31]:


def greedy_random(lay, step_limit=10,verbose = False):
   while lay.steps < step_limit and lay.bad_placed_elements()>0:


    if verbose == True and lay.steps == 0:
      if lay.bad_placed_elements()==0:
        print("No hay elementos mal posicionados")
      print("entrada: ")
      print(lay)

    #numeros random
    i = random.randrange(0,lay.S)
    j = random.randrange(0,lay.S)


    #SI EL MOVIMIENTO ES BUENOBUENO O MALOBUENO
    if lay.is_sorted(i) and lay.is_sorted(j) and lay.g(i)<=lay.g(j):
      lay.move((i,j))
      if verbose == True:
        print("Indices aleatorios: (",i,",",j,")")
        print("El movimientos es bueno-bueno")
        print(lay)
        print("Elementos mal posicionados: ",lay.bad_placed_elements())

    elif not lay.is_sorted(i) and lay.is_sorted(j) and lay.g(i)<=lay.g(j):
      lay.move((i,j))
      if verbose == True:
        print("Indices aleatorios: (",i,",",j,")")
        print("El movimientos es malo-bueno")
        print(lay)
        print("Elementos mal posicionados: ",lay.bad_placed_elements())
    else:
      if verbose == True: 
        print("Indices aleatorios: (",i,",",j,")")
        print("no se recomienda hacer ese tipo de movimiento\n")

    if lay.bad_placed_elements()==0:
       if (verbose==True): 
         print("No hay mas elementos mal posicionados")
       return True, lay.steps
   
   return False, lay.steps

In [38]:
lay = generate_random_layout(5,5,7)

print("entrada: \n",lay)
print(lay.whereMax())
greedy_random(lay,10,False)
print("salida:\n",lay)

entrada: 
 stack 0: [4]
stack 1: [7]
stack 2: [4, 2]
stack 3: [3, 3, 4]
stack 4: []

(1, 0, 7)
salida:
 stack 0: [4, 2]
stack 1: [4]
stack 2: [4]
stack 3: [3, 3]
stack 4: [7]



In [10]:
lay = generate_random_layout(5,5,7)
print("entrada: \n",lay)
greedy_random(lay,10,False)
print("salida:\n",lay)

entrada: 
 stack 0: [1, 2, 6]
stack 1: [6, 2]
stack 2: [1]
stack 3: [7]
stack 4: []

salida:
 stack 0: [1]
stack 1: [6, 2]
stack 2: []
stack 3: [7, 6, 1]
stack 4: [2]



##2.2 Descripcion del Algoritmo
**Entrada**: Un layout (lista de stacks de contenedores) y una altura máxima $H$.

**Salida**: **Mínima secuencia de movimientos** que transformen el layout de entrada en un **layout factible**.

El algoritmo en ordenar los containers de tal manera que los contenedores con una mayor prioridad puedan quedar con un facil acceso.


1. Se asignan indices aleatorios $(i,j)$, para luego utilizar estos indices en los movimientos

El aloritmo solo realizará movimientos, bueno-bueno y malo-bueno, esto quiere decir que no moverá containers que estén bien posicionados a una posicion donde queden mal posicionados

2. Se comprueba que los movimientos haran movimientos **certeros**

3. Se repite el proceso hasta que la cantidad de movimientos maximos se haya excedido o 



##2.3 Ejemplo

Entrada:

```
stack 0: [1]
stack 1: [3]
stack 2: [3, 3]
stack 3: [2, 3]
stack 4: [2]
```
* Se asignan indices Aleatorios


```
Indices aleatorios: ( 0 , 2 )
```
* Se comprueba que los indices no hagan movimientos ineficientes
* Si es asi, se hacen los movimientos, de no ser así se buscan nuevos 



```
stack 0: []
stack 1: [3]
stack 2: [3, 3, 1]
stack 3: [2, 3]
stack 4: [2]
```

* Se asignan nuevos indices aleatorios: $( 4 , 4 )$ y se comprueba que el movimiento es Bueno-Bueno



```
stack 0: []
stack 1: [3]
stack 2: [3, 3, 1]
stack 3: [2, 3]
stack 4: [2]
```

* A esta altura de la ejecucion, solo queda 1 elemento mal posicionado, que es el 3 del stack 3
* Se asignan nuevos valores aleatorios $( 0 , 1 )$



```
stack 0: []
stack 1: [3]
stack 2: [3, 3, 1]
stack 3: [2, 3]
stack 4: [2]
```

* Indices aleatorios: ( 2 , 3 ) no se recomienda hacer ese tipo de movimiento

* Indices aleatorios: ( 4 , 2 ) no se recomienda hacer ese tipo de movimiento

* Indices aleatorios: ( 1 , 2 ) no se recomienda hacer ese tipo de movimiento

* Indices aleatorios: ( 3 , 1 )

```
stack 0: []
stack 1: [3, 3]
stack 2: [3, 3, 1]
stack 3: [2]
stack 4: [2]
```

* En este punto de la ejecucion el layout tiene 0 containers mal posicionados, por lo que retorna True y la cantidad de movimientos que realizó



## 2.4 Ejemplo (Verbose = True)

In [4]:
lay = generate_random_layout(5,5,7)
greedy_random(lay,10,True)

entrada: 
stack 0: [4, 1]
stack 1: [2, 7]
stack 2: [5]
stack 3: [7]
stack 4: [2]

Indices aleatorios: ( 4 , 4 )
El movimientos es bueno-bueno
stack 0: [4, 1]
stack 1: [2, 7]
stack 2: [5]
stack 3: [7]
stack 4: [2]

Elementos mal posicionados:  1
entrada: 
stack 0: [4, 1]
stack 1: [2, 7]
stack 2: [5]
stack 3: [7]
stack 4: [2]

Indices aleatorios: ( 2 , 4 )
no se recomienda hacer ese tipo de movimiento

entrada: 
stack 0: [4, 1]
stack 1: [2, 7]
stack 2: [5]
stack 3: [7]
stack 4: [2]

Indices aleatorios: ( 3 , 0 )
no se recomienda hacer ese tipo de movimiento

entrada: 
stack 0: [4, 1]
stack 1: [2, 7]
stack 2: [5]
stack 3: [7]
stack 4: [2]

Indices aleatorios: ( 0 , 1 )
no se recomienda hacer ese tipo de movimiento

entrada: 
stack 0: [4, 1]
stack 1: [2, 7]
stack 2: [5]
stack 3: [7]
stack 4: [2]

Indices aleatorios: ( 1 , 0 )
no se recomienda hacer ese tipo de movimiento

entrada: 
stack 0: [4, 1]
stack 1: [2, 7]
stack 2: [5]
stack 3: [7]
stack 4: [2]

Indices aleatorios: ( 1 , 4 )
no se r

(True, 2)

## 2.5 Fallas del Algoritmo

El algoritmo Puede fallar cuando tiene un container de menor prioridad mas arriba, esto se puede solucionar buscando la posicion del container y hacer movimientos para dejarlo en la base algun stack, idealmente el stack con menos altura (para hacer menos movimientos).

# Experimentos

In [306]:
def validate(greedy, S=5,H=5,N=10, step_limit=10, n_problems=100, seed=1234):
  random.seed(seed)
  solved = 0; total_steps = 0
  for i in range(n_problems):
    lay = generate_random_layout(S,H,N)
    ret, steps = greedy(lay, step_limit=step_limit)
    if ret: solved+=1
    total_steps += steps
  return solved, total_steps/n_problems

solved, av_steps = validate(greedy_random, S=5,H=5,N=10, step_limit=20, n_problems=100, seed=1234)
print("instancias resueltas:", solved, "/", 100)
print("pasos promedio:", av_steps)

instancias resueltas: 66 / 100
pasos promedio: 13.73


In [307]:
def greedy_randomProfe(lay, step_limit=10):
   while lay.steps < step_limit and lay.bad_placed_elements()>0:
    i = random.randrange(0,lay.S)
    j = random.randrange(0,lay.S)
    lay.move((i,j))

   if lay.bad_placed_elements()==0: return True, lay.steps
   return False, lay.steps

def validate(greedy, S=5,H=5,N=10, step_limit=10, n_problems=100, seed=1234):
  random.seed(seed)
  solved = 0; total_steps = 0
  for i in range(n_problems):
    lay = generate_random_layout(S,H,N)
    ret, steps = greedy(lay, step_limit=step_limit)
    if ret: solved+=1
    total_steps += steps
  return solved, total_steps/n_problems

solved, av_steps = validate(greedy_randomProfe, S=5,H=5,N=10, step_limit=20, n_problems=100, seed=1234)
print("instancias resueltas:", solved, "/", 100)
print("pasos promedio:", av_steps)

instancias resueltas: 6 / 100
pasos promedio: 19.48


El Algoritmo que solo hace movimientos aleatorios tiene una precision de 0.06% ya que alcanza a resolver muy pocos problemas, en cambio cuando ponemos las condicones de solo hacer movimientos buenos/buenos y malos/buenos hace solo movimientos que se acercan a la solucion y no que se alejen de ella.


In [308]:

def greedy_random2(lay, step_limit=10):  
  if lay.bad_placed_elements()==0: return True, lay.steps
  while lay.steps < step_limit and lay.bad_placed_elements()>0:
    sMax,fMax,num = lay.whereMax()
    if fMax != 0: #Se comprueba que container mas pesado no se encuentre abajo
      if lay.isMaxTop():#se comprueba que el conteiner mas pesado se encuentre arriba, en ese caso hay que buscar un stack vacio o vaciar alguno
        if lay.whereEmpty() != -1:#se busca un stack vacio, de no ser así se tendrá que vaciar alguno
          lay.move((sMax,lay.whereEmpty()))
        else:#se busca vaciar Stack
          idx = lay.emptyingStack(sMax) #funcion que vacia un stack aleatorio sin modificar el stack de entrada
      else: 
        while lay.g(sMax) != num: #mientras el valor que esta mas arriba del stack no sea el numero mayor
          i = random.randrange(0,lay.S)
          if lay.whereEmpty() != -1:
            while i == sMax or i == lay.whereEmpty():#se empieza a quitar los containers que estan arriba del container con mayor valor y no moviendolos a un stack vacio
              i = random.randrange(0,lay.S) 
            lay.move((sMax,i))
          else: 
            while i == sMax:#se empieza a quitar los containers que estan arriba del container con mayor valor y moviendolos a un stack no vacio
              i = random.randrange(0,lay.S) 
            lay.move((sMax,i))
        idx=lay.emptyingStack(sMax)
        lay.move((sMax,idx))
          
    i = random.randrange(0,lay.S)
    j = random.randrange(0,lay.S)
    if lay.is_sorted(i) and lay.is_sorted(j) and lay.g(i)<=lay.g(j):
      lay.move((i,j))
    elif not lay.is_sorted(i) and lay.is_sorted(j) and lay.g(i)<=lay.g(j):
      lay.move((i,j))

    if lay.bad_placed_elements()==0: return True, lay.steps
    return False, lay.steps


solved, av_steps = validate(greedy_random2, S=5,H=5,N=10, step_limit=20, n_problems=100, seed=1234)
print("instancias resueltas:", solved, "/", 100)
print("pasos promedio:", av_steps)

instancias resueltas: 7 / 100
pasos promedio: 0.88


En este caso se hicieron condiciones si encontraba el container con menor prioridad en una posicion que no fuese la base se hacian los cambios respectivos para hacer que ese container se mantenga en la base, ahora el problema es que la funcion se relizó pensando en que solo habría un contendor con la minima prioridad, resulta que esto no es asi, al encontrarse mas de un container con la prioridad minima, se empiezan a hacer movimientos innecesario, haciendo que los movimientos se agoten y no los pueda resolver. 

Sin embargo los problemas que tienen solo un container de minima prioridad, se resuelven de manera mas rapida

En este caso resolvio 7 de 100 con pasos promedio de 0.88, esti quiere decir que los problemas que resolvió se hicieron movimientos muy precisos, y que tambien hubo casos(dentro de esos 7) en que los problemas ya estaban resueltos